In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras import optimizers
import os
#from PIL import Image

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import glob
frame_filename = sorted(glob.glob("./frame_level/Train/*.tfrecord"))

In [ ]:
feat_rgb = []
#feat_audio = []
labels = []
j = 0

In [ ]:
import time
frame_lvl_record = frame_filename[-2]
start_time = time.time()
sess = tf.InteractiveSession()
for example in tf.python_io.tf_record_iterator(frame_lvl_record):  
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    if n_frames < 120:
        continue
    labels.append(tf_seq_example.context.feature['labels'].int64_list.value)
    rgb_frame = []
    #audio_frame = []
    # iterate through frames
    #for i in range(n_frames):
    for i in range(120):
        i = int(i*n_frames/120)
        rgb_frame.append(tf.cast(tf.decode_raw(tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8), tf.float32).eval())
        #audio_frame.append(tf.cast(tf.decode_raw(tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8), tf.float32).eval())
    
    feat_rgb.append(rgb_frame)
    #feat_audio.append(audio_frame)
    j += 1
    if j > 10:
        break
sess.close()
print(time.time() - start_time)

In [ ]:
new_rgb = []
for video in feat_rgb:
    new_frame = []
    for frame in video:
        #for i in range(len(img)):
        #    img[i] = (img[i] - np.min(img))/(np.max(img) - np.min(img))
        new_frame.append(np.reshape(frame, (32, 32, 1)))
    new_rgb.append(np.array(new_frame))

In [ ]:
new_labels = []
for label_list in labels:
    label_one_hot = np.zeros(4716)
    for label in label_list:
        label_one_hot[label] = 1
    new_labels.append(label_one_hot)

In [ ]:
new_rgb = np.array(new_rgb)

In [ ]:
new_labels = np.array(new_labels)

In [ ]:
feed_data = []
for i in range(50):
    feed_data.append((new_rgb[i], new_labels[i]))

In [ ]:
np.save("frames.npy", new_rgb)
np.save("labels.npy", new_labels)    

In [3]:
new_rgb = np.load("frames.npy")
new_labels = np.load("labels.npy")

In [ ]:
labels.shape

In [4]:
new_labels = new_labels[4972:]

In [5]:
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM

In [6]:
#Model
model = Sequential()

model.add(TimeDistributed(Conv2D(32, (7,7), strides=(1, 1), activation='relu', 
                                 padding='same'), input_shape=(120, 32, 32, 1)))
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

model.add(TimeDistributed(Flatten())) #flatten feature map into 1 dimension

model.add(Dropout(0.2)) #Dropout to prevent overfitting
model.add(LSTM(256, return_sequences=False, dropout=0.2))
model.add(Dense(4716, activation='sigmoid'))

In [7]:
#perform gradient descent
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [8]:
from random import randint
def generator():
    while 1:
        video = []
        label = []
        for _ in range(32):
            i = randint(0, 320)
            #yield (np.array(new_rgb[i]), np.array(new_labels[i]))
            video.append(new_rgb[i])
            label.append(new_labels[i])
        yield (np.array(video), np.array(label))

In [10]:
model.fit_generator(generator(), steps_per_epoch = 11, epochs = 20)
#model.fit(new_rgb, new_labels, epochs = 5, validation_data = (new_rgb, new_labels))

Epoch 1/20
11/11 [==============================] - 582s 53s/step - loss: 26.7735 - acc: 0.1562
Epoch 2/20
11/11 [==============================] - 586s 53s/step - loss: 25.4513 - acc: 0.1875
Epoch 3/20
11/11 [==============================] - 580s 53s/step - loss: 22.5470 - acc: 0.2273
Epoch 4/20
11/11 [==============================] - 580s 53s/step - loss: 20.0837 - acc: 0.1847
Epoch 5/20
11/11 [==============================] - 591s 54s/step - loss: 20.4065 - acc: 0.1903
Epoch 6/20
11/11 [==============================] - 592s 54s/step - loss: 20.0445 - acc: 0.2045
Epoch 7/20
11/11 [==============================] - 590s 54s/step - loss: 18.9169 - acc: 0.1733
Epoch 8/20
11/11 [==============================] - 602s 55s/step - loss: 20.0722 - acc: 0.1705
Epoch 9/20
11/11 [==============================] - 612s 56s/step - loss: 17.4560 - acc: 0.2188
Epoch 10/20
11/11 [==============================] - 612s 56s/step - loss: 18.6893 - acc: 0.1818
Epoch 11/20
11/11 [====================

KeyboardInterrupt: 

In [ ]:
model.summary()

In [17]:
score = model.evaluate(new_rgb, new_labels)

330/330 [==============================] - 86s 262ms/step


In [18]:
score

[18.466948561234908, 0.19696969701485201]

In [20]:
labels = []
for new_label in new_labels:
    label = []
    for i in range(4716):
        if new_label[i] == 1:
            label.append(i)
    labels.append(label)

In [11]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model_lrcn_330.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model_lrcn_330.h5')

In [ ]:
json_file = open('model_adam_0_7_mil.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_adam_0_7_mil.h5")
print("Loaded model from disk")
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [12]:
predicted_labels = model.predict(new_rgb)

In [ ]:
len(labels)

In [ ]:
count = 0
total = len(labels)
for j in range(total):
    actual_len = len(labels[j])

    temp_list = []
    for i in range(len(predicted_labels[j])):
        temp_list.append((i, predicted_labels[j][i]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    labels_common = 0
    for idx, _ in temp_list[:actual_len]:
        if idx in labels[j]:
            labels_common += 1
    #print(labels_common, actual_len/2)
    if labels_common >= actual_len/2:
        count += 1
        #print(count)
print(count*100/total)

In [ ]:
count = 0
#total = len(labels_val)
total = 0
for j in range(len(labels)):
    actual_len = len(labels[j])

    temp_list = []
    for i in range(len(predicted_labels[j])):
        temp_list.append((i, predicted_labels[j][i]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    labels_common = 0
    for idx, _ in temp_list[:actual_len]:
        if idx in labels[j]:
            labels_common += 1
    #print(labels_common, actual_len/2)
    #if labels_common >= actual_len/2:
    count += labels_common
    total += actual_len
        #print(count)
print(count*100/total)

In [13]:
import pandas as pd
df = pd.DataFrame.from_csv("vocabulary.csv")

In [14]:
vid_list = df["Name"]

In [21]:
#Training set demo
k = 10
labels[k]

[6, 116]

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(vid_ids[k])

In [22]:
print("Actual Labels are: ")
for label in labels[k]:
    print(vid_list[label])
actual_len = len(labels[k])
print("\n\nPredicted Labels are: ")

temp_list = []
for i in range(len(predicted_labels[k])):
    temp_list.append((i, predicted_labels[k][i]))
temp_list.sort(key = lambda x: x[1], reverse = True)
for idx, _ in temp_list[:2*actual_len]:
    print(vid_list[idx])
    
labels_common = 0
for idx, _ in temp_list[:2*actual_len]:
    if idx in labels[k]:
        labels_common += 1
print("\n\nNumber of labels that were found common : "+str(labels_common) + " out of " + str(actual_len))

Actual Labels are: 
Animation
Pokémon


Predicted Labels are: 
Game
Video game
Vehicle
Concert


Number of labels that were found common : 0 out of 2


In [ ]:
video_filenames_val = sorted(glob.glob("./CS 688/Project/Y8M/video_level/Validate/*.tfrecord"))

vid_ids_val = []
labels_val = []
mean_rgb_val = []
mean_audio_val = []
for vfn in video_filenames_val[:3]:
    for example in tf.python_io.tf_record_iterator(vfn):
        tf_example = tf.train.Example.FromString(example)
        vid_ids_val.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
        labels_val.append(tf_example.features.feature['labels'].int64_list.value)
        mean_rgb_val.append(tf_example.features.feature['mean_rgb'].float_list.value)
        #mean_audio_val.append(tf_example.features.feature['mean_audio'].float_list.value)
        
new_rgb_val = []
for img in mean_rgb_val:
    #for i in range(len(img)):
    #    img[i] = (img[i] - np.min(img))/(np.max(img) - np.min(img))
    new_img = np.reshape(img, (32, 32, 1))
    new_rgb_val.append(new_img)
    
new_rgb_val = np.array(new_rgb_val)

In [ ]:
predicted_labels_val = model.predict(new_rgb_val)

In [ ]:
new_labels_val = []
for label_list in labels_val:
    label_one_hot = np.zeros(4716)
    for label in label_list:
        label_one_hot[label] = 1
    new_labels_val.append(label_one_hot)
new_labels_val = np.array(new_labels_val)

In [ ]:
new_labels_val.shape

In [ ]:
#Validation set demo
k = 12
labels_val[k]

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(vid_ids_val[k])

In [ ]:
print("Actual Labels are: ")
for label in labels_val[k]:
    print(vid_list[label])
actual_len = len(labels_val[k])
print("\n\nPredicted Labels are: ")

temp_list = []
for i in range(len(predicted_labels_val[k])):
    temp_list.append((i, predicted_labels_val[k][i]))
temp_list.sort(key = lambda x: x[1], reverse = True)
for idx, _ in temp_list[:2*actual_len]:
    print(vid_list[idx])
    
labels_common = 0
for idx, _ in temp_list[:2*actual_len]:
    if idx in labels_val[k]:
        labels_common += 1
print("\n\nNumber of labels that were found common : "+str(labels_common) + " out of " + str(actual_len))

In [ ]:
count = 0
total = len(labels_val)
for j in range(total):
    actual_len = len(labels_val[j])

    temp_list = []
    for i in range(len(predicted_labels_val[j])):
        temp_list.append((i, predicted_labels_val[j][i]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    labels_common = 0
    for idx, _ in temp_list[:2*actual_len]:
        if idx in labels_val[j]:
            labels_common += 1
    #print(labels_common, actual_len/2)
    if labels_common >= actual_len/2:
        count += 1
        #print(count)
print(count*100/total)

In [ ]:
count = 0
#total = len(labels_val)
total = 0
for j in range(len(labels_val)):
    actual_len = len(labels_val[j])

    temp_list = []
    for i in range(len(predicted_labels_val[j])):
        temp_list.append((i, predicted_labels_val[j][i]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    labels_common = 0
    for idx, _ in temp_list[:actual_len]:
        if idx in labels_val[j]:
            labels_common += 1
    #print(labels_common, actual_len/2)
    #if labels_common >= actual_len/2:
    count += labels_common
    total += actual_len
        #print(count)
print(count*100/total)

In [ ]:
#Calculates the Global Average Precision by Google
print("Global Average Precision for validation Data : ")
N = 20
gap = 0
for i in range(len(labels_val)):
    actual_len = len(labels_val[i])
    temp_list = []
    for j in range(len(predicted_labels_val[i])):
        temp_list.append((j, predicted_labels_val[i][j]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    num_pos = min(actual_len, N)

    delta_recall = 1.0/num_pos
    
    ap = 0
    labels_common = 0
    N = min(actual_len, N)
    for j in range(N):
        temp_label, _ = temp_list[j]
        if temp_label in labels_val[i]: 
            labels_common += 1
        ap += (float(labels_common)*delta_recall)/(j+1)
    gap += ap
print(gap/len(labels_val))

In [ ]:
len(labels)

In [23]:
#Calculates the Global Average Precision by Google
print("Global Average Precision for training Data : ")
N = 20
gap = 0
for i in range(len(labels[:201])):
    actual_len = len(labels[i])
    temp_list = []
    for j in range(len(predicted_labels[i])):
        temp_list.append((j, predicted_labels[i][j]))
    temp_list.sort(key = lambda x: x[1], reverse = True)

    num_pos = min(actual_len, N)

    delta_recall = 1.0/num_pos
    
    ap = 0
    labels_common = 0
    N = min(actual_len, N)
    for j in range(N):
        temp_label, _ = temp_list[j]
        if temp_label in labels[i]: 
            labels_common += 1
        ap += (float(labels_common)*delta_recall)/(j+1)
    gap += ap
print(gap/len(labels))

Global Average Precision for training Data : 
0.11539186507936507
